In [1]:
import pandas as pd
import numpy as np
import glob
import os
import torch.nn.functional as F
import torch

need to do 20201129_20210201 third party attack
20200813_20201025 all

In [2]:
dir = './data/comments'
datasets_dir = "./data/datasets"

files = sorted(glob.glob(f'{dir}/*.pickle'))

print(files)

['./data/comments/20200201_20200315_all_comments_with_labels.pickle', './data/comments/20200315_20200528_all_comments_with_labels.pickle', './data/comments/20200528_20200713_all_comments_with_labels.pickle', './data/comments/20200713_20200813_all_comments_with_labels.pickle', './data/comments/20200813_20201025_all_comments_with_labels.pickle', './data/comments/20201025_20201128_all_comments_with_labels.pickle', './data/comments/20201129_20210201_all_comments_with_labels.pickle', './data/comments/all_comments.pickle', './data/comments/all_comments_no_liwc.pickle']


In [3]:
fname = files[6]
date = fname.split('/')[-1].split('_all_comments')[0]
print(fname)
df = pd.read_pickle(fname)
df

./data/comments/20201129_20210201_all_comments_with_labels.pickle


,created_utc,parent_id,link_id,id,author,body,subreddit,link,score,author_flair_text,...,social,posemo,negemo,sad,we,anger,swear,i,you,stereotype
1,1612153203,t3_l9gngx,t3_l9gngx,glk8hec,runrain,Lol not even. Where I heard about it most was ...,AskThe_Donald,/r/AskThe_Donald/comments/l9gngx/doj_arrest_re...,1,:T: NOVICE,...,7,6,3,1,2,0,0,0,0,0.234234
2,1612152460,t3_l9t1ha,t3_l9t1ha,glk757a,SilverHerfer,To paraphrase one of the great conservatives o...,AskThe_Donald,/r/AskThe_Donald/comments/l9t1ha/we_definitely...,3,:T: NOVICE,...,2,2,0,0,1,0,0,0,0,0.536770
4,1612151108,t1_glit0la,t3_l9lonl,glk4jhg,Psychowitz,Like coding.,AskThe_Donald,/r/AskThe_Donald/comments/l9lonl/keystone_pipe...,0,:T: NOVICE,...,0,0,0,0,0,0,0,0,0,0.288935
6,1612150714,t3_l9t1ha,t3_l9t1ha,glk3rkp,Salmankhan42069,"Wait, this is a meme?",AskThe_Donald,/r/AskThe_Donald/comments/l9t1ha/we_definitely...,-1,:T: NOVICE,...,0,0,0,0,0,0,0,0,0,0.249884
9,1612150618,t3_l9t1ha,t3_l9t1ha,glk3kvz,Pile_of_Walthers,Which one is more useful?,AskThe_Donald,/r/AskThe_Donald/comments/l9t1ha/we_definitely...,2,:T: NOVICE,...,0,1,0,0,0,0,0,0,0,0.308906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7953553,1606658031,t1_ge13eq1,t3_k37m33,ge15l1b,The_Countryof_Poland,hmmm you’re right. the deep state goes even de...,VoteDEM,/r/VoteDEM/comments/k37m33/daily_discussion_th...,1,NaN,...,1,0,0,0,0,0,0,1,1,0.495541
7953555,1606657109,t3_k37m33,t3_k37m33,ge13eq1,arthurpenhaligon,Trump has lost essentially every lawsuit in th...,VoteDEM,/r/VoteDEM/comments/k37m33/daily_discussion_th...,1,:Missouri: Missouri,...,3,1,3,1,0,0,0,0,0,0.332276
7953561,1606653320,t3_k37m33,t3_k37m33,ge0v2ee,screen317,"Last night we passed 20,000 VoteDEM members! C...",VoteDEM,/r/VoteDEM/comments/k37m33/daily_discussion_th...,1,:New_Jersey: NJ-12,...,2,0,0,0,1,0,0,0,0,0.401069
7953562,1606653034,t3_k37m33,t3_k37m33,ge0uf7z,Jeffery_G,The wife and her ladies group produced 200+ po...,VoteDEM,/r/VoteDEM/comments/k37m33/daily_discussion_th...,1,:Georgia: Georgia,...,4,2,0,0,0,0,0,0,0,0.289892


In [4]:
df.columns

Index(['created_utc', 'parent_id', 'link_id', 'id', 'author', 'body',
       'subreddit', 'link', 'score', 'author_flair_text', 'vulgarity',
       'personal_attacks', 'incivility', 'third_party_attacks', 'aspersion',
       'comment_len', 'compound_sentiment', 'categories', 'negate', 'social',
       'posemo', 'negemo', 'sad', 'we', 'anger', 'swear', 'i', 'you',
       'stereotype'],
      dtype='object')

In [6]:
categories = ['aspersion', 'civility', 'vulgarity', 'personal_attack', 'third_party_attack', 'stereotype']
files = ['', '_first', '_second', '_third']
for c in categories:
    print(c)
    labels = []
    for f in files:
        print(f)
        path = f'data/labels/{date}_all_comments_with_labels{f}_roberta/{c}.npy'
        if os.path.isfile(path):
            print("there exists dataset under path " + path)
            with open(path, 'rb') as f:
                labels_dict = np.load(f, allow_pickle=True)
                acts = (F.softmax(torch.tensor(np.stack(labels_dict)))).cpu().detach().numpy()[:,1]
                print(len(acts))
                labels.extend(acts)
        else:
            print("no dataset under path " + path)
    df[c] = labels
df

aspersion

no dataset under path data/labels/20201129_20210201_all_comments_with_labels_roberta/aspersion.npy
_first
there exists dataset under path data/labels/20201129_20210201_all_comments_with_labels_first_roberta/aspersion.npy


/Users/yujia/opt/anaconda3/envs/incivility/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


3000000
_second
there exists dataset under path data/labels/20201129_20210201_all_comments_with_labels_second_roberta/aspersion.npy
2000000
_third
there exists dataset under path data/labels/20201129_20210201_all_comments_with_labels_third_roberta/aspersion.npy
1786187
civility

no dataset under path data/labels/20201129_20210201_all_comments_with_labels_roberta/civility.npy
_first
there exists dataset under path data/labels/20201129_20210201_all_comments_with_labels_first_roberta/civility.npy
3000000
_second
there exists dataset under path data/labels/20201129_20210201_all_comments_with_labels_second_roberta/civility.npy
2000000
_third
there exists dataset under path data/labels/20201129_20210201_all_comments_with_labels_third_roberta/civility.npy
1786187
vulgarity

no dataset under path data/labels/20201129_20210201_all_comments_with_labels_roberta/vulgarity.npy
_first
there exists dataset under path data/labels/20201129_20210201_all_comments_with_labels_first_roberta/vulgarity.npy
3

ValueError: Length of values (5094720) does not match length of index (6786187)

In [ ]:
df.to_pickle(f'data/roberta/{date}.pickle')